In [1]:
#!pip install numpy==1.19.3
#!pip install pandas==1.1.4
#!pip install selenium==3.141.0
#!pip install openpyxl
#!pip install psycopg2

In [2]:
import pandas as pd
import numpy as np
import time
import datetime
import psycopg2
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
# chrome_options.add_argument("--incognito")
chrome_options.add_argument("--start-maximized")

## Lấy thông tin của một sản phẩm

In [3]:
def get_item_info(shop_id, item_id):
    url = f'https://shopee.vn/product/{shop_id}/{item_id}'
    driver.get(url)
#     time.sleep(1)

    # Name
    while True:
        e = driver.find_elements_by_css_selector('.OSgLcw')
        if e:
            item_name = e[0].text
            break

    # Min Price
    while True:
        e = driver.find_elements_by_css_selector('._3n5NQx')
        if e and e[0].text != '':
            item_price = e[0].text.split()[0]
            item_price = int(item_price[1:].replace('.', ''))
            break

    # Brand
#     while True:
    e = driver.find_elements_by_css_selector('._2H-513')
    if e:
        item_brand_name = e[0].text
    else:
        item_brand_name = 'No Brand'
#             break

    # Last category
    while True:
        e = driver.find_elements_by_css_selector('.JFOy4z')
        if e:
            item_category = int(e[-1].get_attribute('href').split('.')[-1])
            break

    # Sold
    while True:
        e = driver.find_elements_by_css_selector('._22sp0A')
        if e:
            num_of_sold = e[0].text.replace('(', '').replace(')', '').split()[-1].replace(',', '.')
            if num_of_sold[-1] == 'k':
                num_of_sold = int(float(num_of_sold.replace('k', '')) * 1000)
            else:
                num_of_sold = int(num_of_sold.replace('k', ''))
            break
     
    # Favourite
    while True:
        e = driver.find_elements_by_css_selector('._1-aYcb')
        if e:
            if len(e) > 1:
                num_of_fav = e[1].text.replace('(', '').replace(')', '').split()[-1].replace(',', '.')
                if num_of_fav[-1] == 'k':
                    num_of_fav = int(float(num_of_fav.replace('k', '')) * 1000)
                else:
                    num_of_fav = int(num_of_fav.replace('k', ''))
            else:
                num_of_fav = 0
            break

    # Rating
    d = 0
    while True:
        if d == 50:
            if driver.find_elements_by_css_selector('.product-ratings-comments-view__no-data__text'):
                _5_star, _4_star, _3_star, _2_star, _1_star = (0, 0, 0, 0, 0)
                break
            else:
                d = 0

        if not driver.find_elements_by_css_selector('._3Oj5_n'):
            _5_star, _4_star, _3_star, _2_star, _1_star = (0, 0, 0, 0, 0)
            break

        d += 1
        driver.execute_script(f"window.scrollTo(0, {d * 100})")
        e = driver.find_elements_by_css_selector('.product-rating-overview__filter')
        if e:
            e = [i.text.split()[-1].replace('(', '').replace(')', '').replace(',', '.') for i in e[1:6]]
            e = tuple(int(float(i.replace('k', '')) * 1000) if i[-1] == 'k' else int(i.replace('k', '')) \
                    for i in e)
            _5_star, _4_star, _3_star, _2_star, _1_star = e
            break

    return [str(item_id), item_name, item_price, item_brand_name, str(shop_id), item_category, num_of_fav, num_of_sold, _5_star, _4_star, _3_star, _2_star, _1_star]

## Lấy thông tin tất cả sản phẩm của một shop

In [4]:
def get_shop_items_id(shop_id):
    url = f'https://shopee.vn/shop/{shop_id}/search'
    driver.get(url)
    time.sleep(1)

    while True:
        e = driver.find_elements_by_css_selector('.shopee-mini-page-controller__total')
        if e:
            total_pages = int(e[0].text)
            break

    all_items_id = []
    for i in range(total_pages):
        a_page = url + f'?page={i}&sortBy=pop'
        driver.get(a_page)
        time.sleep(1)

        all_items = driver.find_elements_by_xpath('//a[@data-sqe="link"]')
        all_items_id += [i.get_attribute('href').split('.')[-1] for i in all_items]
    return all_items_id

def get_shop_items(shop_id):
    items_id = get_shop_items_id(shop_id)
    return [get_item_info(shop_id, item_id) for item_id in items_id]

def get_a_shop(shop_id):
    data = get_shop_items(shop_id)
    df = pd.DataFrame(data, columns = ['ID', 'Name', 'Price', 'Brand', 'ID_shop', 'Category', 'Num_of_fav', 'Num_of_sold', '5_star', '4_star', '3_star', '2_star', '1_star'])
    df.set_index('ID', inplace=True)
    df.to_excel(f'1_{shop_id}.xlsx')

### Nhập Categories vào database

In [ ]:
# df = pd.read_excel('Category_Complete.xlsx', engine='openpyxl')
# df.astype({'ID':str, 'Name':str, 'Parent':str})

# with psycopg2.connect(host="localhost",
#                       database="CSDL",
#                       user="postgres",
#                       password="phonhay103",
#                       port="5433") as conn:
#     conn.autocommit = True
#     cur = conn.cursor()

#     for i in range(len(df)):
#         row = list(np.asarray(df.iloc[i]))
#         cur.execute('INSERT INTO categories VALUES (%s, %s, %s);', (int(row[0]), row[1], int(row[2])))
#     cur.close()

### Nhập Cities vào database

In [ ]:
# df = pd.read_excel('City_Complete.xlsx', engine='


..................................openpyxl')
# df.astype({'ID':str, 'Name':str})

# with psycopg2.connect(host="localhost",
#                       database="CSDL",
#                       user="postgres",
#                       password="phonhay103",
#                       port="5433") as conn:
#     conn.autocommit = True
#     cur = conn.cursor()

#     for i in range(len(df)):
#         row = list(np.asarray(df.iloc[i]))
#         cur.execute('INSERT INTO cities VALUES (%s, %s);', (row[0], row[1]))
#     cur.close()